# データの読み取り

In [ ]:
from langchain.document_loaders import PyPDFLoader, PyMuPDFLoader

In [ ]:
# ファイルの読み込み
FILE_PATH = './input/fies_mr-y.pdf' # 事前にファイルをアップロードしておく
loader = PyMuPDFLoader(FILE_PATH)
documents = loader.load()

In [ ]:
# documents

# データをチャンクに分割

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

## 分割方法の定義（以下のどちらかを有効に）

In [ ]:
# text_splitter = CharacterTextSplitter(
#     separator = "\n\n",
#     chunk_size=200,
#     chunk_overlap=0
# )

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "$",
        "\n\n",
        "\uff0e",  # 全角「。」
        "\n",
        "\uff0c",  # 全角コンマ
        ".",
        ",",
        " ",
        "",
    ],
    chunk_size=200,
    chunk_overlap=0
)

## チャンクに分割

In [ ]:
documents_chunk = text_splitter.split_documents(documents)

In [ ]:
# documents_chunk

In [ ]:
type(documents_chunk[0])

In [ ]:
# 文字数の確認
for doc in documents_chunk:
    print("文字数：",len(doc.page_content))

# ベクトルDB作成

## 埋め込みモデル定義

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
# 埋め込みモデルの定義
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
embeddings

In [ ]:
type(embeddings)

## ベクトルDB作成

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
# ベクトルデータベースを作る（ここを2回実行すると、1つのベクトルデータベースに同じデータが2度格納されることになるので注意）
vectordb = Chroma.from_documents(
    documents=documents_chunk,
    embedding=embeddings
)

In [ ]:
vectordb

## 検索確認

In [ ]:
sc = vectordb.similarity_search_with_relevance_scores(query="消費支出の推移はどのようになっていますか？", k=3)
for s in sc:
    print('--')
    print(s)

In [ ]:
questions =[
    {"question":"2023年12月分において、二人以上の世帯における1世帯あたりの消費支出は？"},
    {"question":"2023年12月分において、消費支出（二人以上の世帯）は、前年同月比で、実質的に何%変化しましたか？"},
    {"question":"2023年3月の消費支出は、前年同月の消費支出に比べて、どれくらい変化しましたか？"},
    {"question":"2023年12月分において、消費支出の内訳（2023年12月－二人以上の世帯）に関して、家具・家事用品の実質増減率への寄与度はいくらですか？"},
    {"question":"2023年12月分において、消費支出の内訳（2023年12月－二人以上の世帯）に関して、住居の金額はいくらですか？"},
    {"question":"2023年12月分において、勤労者世帯の実収入（二人以上の世帯）は、前年同月比で、名目的に何%減少しましたか？"},
    {"question":"2023年12月分において、勤労者世帯の収支の内訳に関して、実質増減率への寄与度の絶対値が最も大きい項目は何ですか？値ではなく項目を教えてください"},
    {'question':'第4表 主要家計指標（エクセル：53KB） から、総世帯の消費支出の列を抜き出し、2019年度平均の値を取り出してください'},
    {'question':'第4表 主要家計指標（エクセル：53KB） から、総世帯の世帯人員の列を抜き出し、2019年度平均の値を取り出してください'},
    {"question":"2023年10～12月期平均において、総世帯における1世帯あたりの消費支出は？"},
    {'question':'2023年10～12月期平均において、消費支出（総世帯）は、前年同期比で実質的に何%変化しましたか？'},
    {"question":"2023年10～12月期平均において、勤労者世帯の実収入（二人以上の世帯）は、前年同期比で、実質的に何%減少しましたか？また、名目的に何%減少しましたか？",},
    {"question":"2023年12月分において、勤労者世帯の収支の内訳に関して、世帯主収入の金額と配偶者の収入の金額では、どちらが大きいですか？"},
    {"question":"2023年平均において、総世帯における1世帯あたりの消費支出は？"},
    {"question":"2023年平均において、消費支出の内訳に関して、教養娯楽の金額はいくらですか？"},
    {"question":"2024年の7~9月期平均の家計調査結果の公表日はいつですか？"},
    {"question":"家計調査の消費支出には、外国人観光客による消費分は含まれているか？"},
    {"question":"労働者１人当たりの平均賃金を知りたい場合はどの調査結果を参照すればいいか？"},
    {"question":"総務省統計局統計調査部消費統計課審査発表係の電話番号を教えてください"},
    {"question":"統計メールニュースを申し込むためのURLを教えてください"},
]

In [ ]:
for i, q in enumerate(questions):
    # questions[i]["chunks"] = "\n".join(vectordb.similarity_search_with_relevance_scores(query=q['question'], k=3))
    sc = vectordb.similarity_search_with_relevance_scores(query=q['question'], k=3)
    documents = []
    for s in sc:
        # print('--')
        # print(type(s[0]))
        document, score = s
        # print(document.page_content)
        documents.append("{} : {}".format(document.page_content, score))

    q['documents'] = '\n'.join(documents)

In [ ]:
import pandas as pd

In [ ]:
questions_df = pd.DataFrame(questions)

In [ ]:
questions_df

In [ ]:
import datetime
questions_df.to_csv('output/chromadb-ex01-questions.{}.csv'.format(datetime.datetime.now().strftime('%Y%m%d-%H%M%S')), index=False)